In [14]:
from pathlib import Path
import requests

def download_one_file_of_raw_data(year: int, month: int) -> Path:
    """"""
    URL = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet'
    response = requests.get(URL)

    if response.status_code == 200:
        path = f'../data/raw/rides_{year}-{month:02d}.parquet'
        open(path, "wb").write(response.content)
        return path
    else:
        raise Exception(f'{URL} is not available')

In [15]:
download_one_file_of_raw_data(year=2022, month=1)

'../data/raw/rides_2022-01.parquet'

In [6]:
! pip install pandas

  Using cached pandas-1.5.3-cp39-cp39-win_amd64.whl (10.9 MB)
     ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
     --------------------------------------- 0.0/14.9 MB 660.6 kB/s eta 0:00:23
     --------------------------------------- 0.0/14.9 MB 660.6 kB/s eta 0:00:23
     --------------------------------------- 0.0/14.9 MB 245.8 kB/s eta 0:01:01
     --------------------------------------- 0.0/14.9 MB 245.8 kB/s eta 0:01:01
     --------------------------------------- 0.1/14.9 MB 403.5 kB/s eta 0:00:37
     --------------------------------------- 0.1/14.9 MB 385.0 kB/s eta 0:00:39
     --------------------------------------- 0.1/14.9 MB 385.0 kB/s eta 0:00:39
     --------------------------------------- 0.1/14.9 MB 266.9 kB/s eta 0:00:56
     --------------------------------------- 0.2/14.9 MB 296.2 kB/s eta 0:00:50
      -------------------------------------- 0.2/14.9 MB 436.6 kB/s eta 0:00:34
      -------------------------------------- 0.2/14.9 MB 436.6 kB/

In [12]:
! pip install pyarrow


     ---------------------------------------- 0.0/20.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/20.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/20.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/20.6 MB ? eta -:--:--
     --------------------------------------- 0.1/20.6 MB 302.7 kB/s eta 0:01:08
     --------------------------------------- 0.1/20.6 MB 374.1 kB/s eta 0:00:55
     --------------------------------------- 0.1/20.6 MB 374.1 kB/s eta 0:00:55
     --------------------------------------- 0.1/20.6 MB 374.1 kB/s eta 0:00:55
     --------------------------------------- 0.1/20.6 MB 261.9 kB/s eta 0:01:19
     --------------------------------------- 0.2/20.6 MB 491.0 kB/s eta 0:00:42
     --------------------------------------- 0.2/20.6 MB 491.0 kB/s eta 0:00:42
     --------------------------------------- 0.2/20.6 MB 491.0 kB/s eta 0:00:42
     --------------------------------------- 0.2/20.6 MB 491.0 kB/s eta 

In [16]:
import pandas as pd

rides =  pd.read_parquet('../data/raw/rides_2022-01.parquet')

rides.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [17]:
rides = rides[['tpep_pickup_datetime', 'PULocationID']]
rides.rename(columns={'tpep_pickup_datetime':'pickup_datetime',
                      'PULocationID':'pickup_location_id'},
                      inplace=True)
rides.head()

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68


In [18]:
rides['pickup_datetime'].describe()

C:\Users\gabri\AppData\Local\Temp\ipykernel_3420\3389344848.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  rides['pickup_datetime'].describe()


count                 2463931
unique                1423522
top       2022-01-26 07:57:00
freq                       12
first     2008-12-31 22:23:09
last      2022-05-18 20:41:57
Name: pickup_datetime, dtype: object

In [19]:
rides = rides[rides.pickup_datetime >= '2022-01-01']
rides = rides[rides.pickup_datetime <= '2022-02-01']
rides['pickup_datetime'].describe()

C:\Users\gabri\AppData\Local\Temp\ipykernel_3420\1241092448.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  rides['pickup_datetime'].describe()


count                 2463879
unique                1423471
top       2022-01-26 07:57:00
freq                       12
first     2022-01-01 00:00:08
last      2022-01-31 23:59:58
Name: pickup_datetime, dtype: object

In [ ]:
rides.to_parquet('../data/transformed/validated_rides_2022_01.parquet')